In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, StratifiedKFold
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFECV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from xgboost import XGBRegressor
%matplotlib inline

In [2]:
merge_df = pd.read_csv('../data/merged_fftr.csv')
month_df = pd.read_csv('../data/monthly_data.csv')

In [3]:
merge_df = merge_df.set_index('DATE')
month_df = month_df.set_index('DATE')

In [4]:
merge_df.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_ahead,2_month_ahead,3_month_ahead,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1987-11-09,6.8125,5.773333,17.50,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7930,0.02,134.30,1.140,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-10,6.8125,5.770000,17.75,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7865,0.02,134.55,1.180,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-11,6.8125,5.766667,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7720,0.02,135.50,1.155,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-12,6.8125,5.763333,17.85,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7575,0.02,136.45,1.130,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-13,6.8125,5.760000,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7680,0.02,135.85,1.090,7.3125,7.25,6.625,6.8125,6.8125,6.500


In [5]:
merge_df.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_ahead,2_month_ahead,3_month_ahead,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1987-11-09,6.8125,5.773333,17.50,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7930,0.02,134.30,1.140,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-10,6.8125,5.770000,17.75,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7865,0.02,134.55,1.180,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-11,6.8125,5.766667,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7720,0.02,135.50,1.155,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-12,6.8125,5.763333,17.85,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7575,0.02,136.45,1.130,7.3125,7.25,6.625,6.8125,6.8125,6.625
1987-11-13,6.8125,5.760000,17.80,4.1,6648.1,-0.293283,5007.994,392.107,537.511,60.0022,...,1.7680,0.02,135.85,1.090,7.3125,7.25,6.625,6.8125,6.8125,6.500


In [6]:
merge_df = merge_df.iloc[90:]
merge_df = merge_df.drop(merge_df.tail(90).index)

In [7]:
month_df = month_df.iloc[3:]
month_df = month_df.drop(month_df.tail(3).index)

In [8]:
month_df.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_ahead,2_month_ahead,3_month_ahead,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1988-05-01,6.7500,5.6,16.166667,4.9,6841.7,-0.037442,5190.036,439.471,546.098,60.9952,...,1.873833,0.12,124.94,1.153333,6.7500,6.50,6.625,7.2500,7.5000,7.6875
1988-06-01,7.2500,5.4,16.330000,4.9,6868.5,-0.037442,5190.036,439.471,546.098,61.1697,...,1.823500,0.00,125.20,0.940000,6.7500,6.75,6.500,7.5000,7.6875,8.1250
1988-07-01,7.5000,5.4,13.950000,4.7,6898.2,-0.230370,5282.835,453.586,552.849,61.1749,...,1.701000,0.06,134.10,0.790000,7.2500,6.75,6.750,7.6875,8.1250,8.1250
1988-08-01,7.6875,5.6,15.600000,4.7,6918.4,-0.230370,5282.835,453.586,552.849,61.4851,...,1.713000,0.04,132.80,0.710000,7.5000,7.25,6.750,8.1250,8.1250,8.1250
1988-09-01,8.1250,5.4,14.150000,4.7,6932.4,-0.230370,5282.835,453.586,552.849,61.2844,...,1.669000,0.02,136.35,0.570000,7.6875,7.50,7.250,8.1250,8.1250,8.3750


In [9]:
X = merge_df.drop(columns = ['1_month_ahead', '2_month_ahead', '3_month_ahead'])
y_1 = merge_df['1_month_ahead']
y_2 = merge_df['2_month_ahead']
y_3 = merge_df['3_month_ahead']
X_month = month_df.drop(columns = ['1_month_ahead', '2_month_ahead', '3_month_ahead'])
y_month_1 = month_df['1_month_ahead']
y_month_2 = month_df['2_month_ahead']
y_month_3 = month_df['3_month_ahead']

In [10]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y_1, random_state=42, test_size=0.25)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y_2, random_state=42, test_size=0.25)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y_3, random_state=42, test_size=0.25)
X_train_month_1, X_test_month_1, y_train_month_1, y_test_month_1 = train_test_split(X_month, y_month_1, random_state=42, test_size=0.25)
X_train_month_2, X_test_month_2, y_train_month_2, y_test_month_2 = train_test_split(X_month, y_month_2, random_state=42, test_size=0.25)
X_train_month_3, X_test_month_3, y_train_month_3, y_test_month_3 = train_test_split(X_month, y_month_3, random_state=42, test_size=0.25)

In [12]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_month_1, y_train_month_1)
print(datetime.now())

2024-02-28 15:19:08.799684
2024-02-28 15:19:31.145773


In [13]:
rfr_CV.best_params_

{'max_depth': 7, 'max_features': 'log2', 'n_estimators': 400}

In [14]:
best_rfr_model = rfr_CV.best_estimator_

In [15]:
prediction = best_rfr_model.predict(X_test_month_1)
mse = mean_squared_error(y_test_month_1, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.10952993091074674
0.33095306451330336


In [16]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [2,3,4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_month_2, y_train_month_2)
print(datetime.now())

2024-02-28 15:19:31.185067
2024-02-28 15:20:13.131597


In [17]:
rfr_CV.best_params_

{'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 500}

In [18]:
best_rfr_model = rfr_CV.best_estimator_

In [19]:
prediction = best_rfr_model.predict(X_test_month_2)
mse = mean_squared_error(y_test_month_2, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.12492292019546325
0.35344436647860616


In [20]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [2,3,4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_month_3, y_train_month_3)
print(datetime.now())

2024-02-28 15:20:13.194527
2024-02-28 15:20:55.817112


In [21]:
rfr_CV.best_params_

{'max_depth': 7, 'max_features': 'sqrt', 'n_estimators': 300}

In [22]:
best_rfr_model = rfr_CV.best_estimator_

In [23]:
prediction = best_rfr_model.predict(X_test_month_3)
mse = mean_squared_error(y_test_month_3, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.17542083811772005
0.41883270898739516


In [24]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_1, y_train_1)
print(datetime.now())

2024-02-28 15:20:55.851574
2024-02-28 15:24:58.349270


In [25]:
best_rfr_model = rfr_CV.best_estimator_

In [26]:
prediction = best_rfr_model.predict(X_test_1)
mse = mean_squared_error(y_test_1, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.009691476349730326
0.09844529622958288


In [27]:
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [4,5,6,7]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_2, y_train_2)
print(datetime.now())

2024-02-28 15:24:58.440022
2024-02-28 15:29:02.139350


In [28]:
best_rfr_model = rfr_CV.best_estimator_

In [29]:
prediction = best_rfr_model.predict(X_test_2)
mse = mean_squared_error(y_test_2, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.010156539912790395
0.10077966021370778


In [44]:
grid = { 
    'n_estimators': [1000, 1200],
    'max_features': ['sqrt','log2'],
    'max_depth' : [13, 14]
}
print(datetime.now())
rfr_CV = GridSearchCV(estimator=RandomForestRegressor(criterion='squared_error'), param_grid=grid, cv= 5)
rfr_CV.fit(X_train_3, y_train_3)
print(datetime.now())

2024-02-28 15:40:17.215113
2024-02-28 15:45:01.279200


In [45]:
best_rfr_model = rfr_CV.best_estimator_

In [46]:
MAE = mean_absolute_error(y_test_3, 
                          y_pred)
print(MAE)

0.011452591954486575


In [47]:
prediction = best_rfr_model.predict(X_test_3)
mse = mean_squared_error(y_test_3, prediction)
rmse = mse**.5
print(mse)
print(rmse)

0.0013773192159274663
0.037112251561007


In [48]:
rfr_CV.best_params_

{'max_depth': 14, 'max_features': 'sqrt', 'n_estimators': 1200}

In [49]:
r2_score(y_test_3, y_pred)

0.999799738896571

In [34]:
X_train_3.head()

,FFTR,UNRATE,Oil,PCE,DispInc,gdp_gap,GDP,EXPGS,IMPGS,ind_prod,...,m2_velo,nasdaq,pci,gbp_dollar,pct_recession,yen_dollar,yield_curve,1_month_back,2_month_back,3_month_back
DATE,,,,,,,,,,,,,,,,,,,,,
1999-05-06,4.75,4.216129,16.4400,1.4,9469.3,1.605682,9526.210,974.597,1222.126,88.6554,...,2.124,2472.2800,2.153318,1.637700,0.00,120.7700,0.3300,4.75,5.00,5.00
2022-04-06,0.50,3.683333,100.8100,4.7,16042.6,-0.339047,25544.273,3044.272,4069.839,102.7953,...,1.179,13888.8200,4.703935,1.308200,0.24,123.7000,0.1100,1.00,1.00,1.75
2011-02-15,0.25,9.000000,102.4800,1.8,12776.6,-3.974206,15351.448,2040.407,2608.884,92.9930,...,1.728,2804.3500,1.117829,1.612800,0.20,83.7900,2.7700,0.25,0.25,0.25
2013-11-19,0.25,6.780000,108.2900,1.7,12998.9,-2.599131,17192.019,2349.240,2783.259,100.1661,...,1.565,3931.5500,1.890963,1.611000,0.04,100.1400,2.4200,0.25,0.25,0.25
2007-05-28,5.25,4.574194,69.6625,1.7,12059.9,0.892907,14402.082,1622.764,2368.569,101.6299,...,1.986,2568.3425,2.774199,1.982175,0.30,121.7725,0.0025,5.25,5.25,5.25


In [35]:
np.sqrt(np.nanmean(np.square(((y_test_3 - prediction) / y_test_3))))*100

4.492041665004043

In [36]:
grid = {'max_depth': [5,6,7], 'learning_rate': [.1, .2],
        'n_estimators': [600, 700]}
xgb_cv = GridSearchCV(estimator = XGBRegressor(objective='reg:squarederror'), scoring='neg_mean_absolute_error',
                  param_grid = grid, cv = 5)
xgb_cv.fit(X_train_3, y_train_3);

In [37]:
xgb_cv.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=700, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [38]:
best_xgb_model = xgb_cv.best_estimator_
best_xgb_model.fit(X_train_3, y_train_3);
y_pred = best_xgb_model.predict(X_test_3)

In [39]:
MAE = mean_absolute_error(y_test_3, 
                          y_pred)

In [40]:
print(MAE)

0.011452591954486575


In [41]:
RMSE = np.sqrt(mean_squared_error(y_test_3,
                                  y_pred))
print(RMSE)

0.036722356811458294


In [42]:
r2_score(y_test_3, y_pred)

0.999799738896571